In [35]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession, functions as F
import re
import numpy as np
import us

In [2]:
startpath ="_Star_Ratings_and_Display_Measures/"
fallpath ="_Star_Ratings_Fall_Release/"
springpath ="_Star_Ratings_Spring_Release/"
cpath="_Part_C"
dpath="_Part_D"
finalpath ="_Report_Card_Master_Table.xlsx"
year = "2014"



In [3]:
firstfive = ["Contract Number","Organization Type","Contract Name","Organization Marketing Name","Parent Organization"]
lasttwo =["Year","Overall"]

In [4]:
y = 2017
path = f"./Data/{y}{startpath}{y}{fallpath}{y}{finalpath}"
dfms = pd.read_excel(path,"Measure_Stars",header=2)
dfms = dfms.iloc[1:]
for x in range(5,len(dfms.keys())):
    dfms =dfms.rename(columns= {dfms.keys()[x] : re.split(r'\d+:',dfms.keys()[x])[0]+re.split(r'\d+:',dfms.keys()[x])[1]})
for x in range(5):
    dfms = dfms.rename(columns={f"Unnamed: {x}":firstfive[x]})
dfms

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Annual Flu Vaccine,C Improving or Maintaining Physical Health,C Improving or Maintaining Mental Health,...,D Beneficiary Access and Performance Problems,D Drug Plan Quality Improvement,D Rating of Drug Plan,D Getting Needed Prescription Drugs,D MPF Price Accuracy,D High Risk Medication,D Medication Adherence for Diabetes Medications,D Medication Adherence for Hypertension (RAS antagonists),D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR
1,E0654,Employer/Union Only Direct Contract PDP,IBT VOLUNTARY EMPLOYEE BENEFITS TRUST,TEAMStar Medicare Part D Prescription Drug Pro...,IBT Voluntary Employee Benefits Trust,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,5,4,5,Not enough data available,4,3,4,3,3
2,E2630,Employer/Union Only Direct Contract PDP,C AND O EMPLOYEES' HOSPITAL ASSOCIATION,C and O Employees Hospital Association,C & O Employees' Hospital Association,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,4,3,5,5,Not enough data available,3,3,4,4,2
3,E3014,Employer/Union Only Direct Contract PDP,PSERS HOP PROGRAM,Pennsylvania Public School Employees Retiremen...,Commonwealth of PA Pub Schools Retirement System,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,4,5,5,Not enough data available,5,4,5,4,3
4,E4744,Employer/Union Only Direct Contract PDP,MODOT/MSHP MEDICAL AND LIFE INSURANCE PLAN,MISSOURI DEPARTMENT OF TRANSPORTATION,Missouri Highways and Transportation Commission,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,4,4,5,Not enough data available,4,4,5,4,3
5,E7316,Employer/Union Only Direct Contract PDP,UNION PACIFIC RAILROAD EMPLOYES HEALTH SYSTEMS,Union Pacific Railroad Employes Health Systems,Union Pacific Railroad Employes Health Systems,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,2,5,2,Not enough data available,3,1,1,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,S7694,PDP,ENVISION INSURANCE COMPANY,EnvisionRx Plus,Rite Aid Corporation,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,3,4,3,3,5,5,2,1,2,3
604,S8067,PDP,AVALON INSURANCE COMPANY,Avalon Insurance Company,Capital BlueCross,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,1,4,3,3,5,5,5,5,4,4
605,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,4,2,2,Not enough data available,2,3,4,3,5
606,S9579,PDP,TRANSAMERICA LIFE INSURANCE COMPANY,Transamerica Life Insurance Company,Aegon N.V.,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,3,5,5,5,3,3,3,3,3


In [5]:
def get_measure_stars(path):
    dfms = pd.read_excel(path,"Measure_Stars",header=2)
    dfms = dfms.iloc[1:]
    for x in range(5,len(dfms.keys())):
        dfms =dfms.rename(columns= {dfms.keys()[x] : re.split(r'\d+:',dfms.keys()[x])[0]+re.split(r'\d+:',dfms.keys()[x])[1]})
    for x in range(5):
        dfms = dfms.rename(columns={f"Unnamed: {x}":firstfive[x]})
    return dfms

In [6]:
def get_domain_stars(path):
    dfds = pd.read_excel(path,"Domain_Stars",header=1)
    return dfds

In [7]:
def get_summary_rating(path):
    dfsr = pd.read_excel(path,"Summary_Rating",header=1)
    dfsr = dfsr.drop(columns="Sanction Deduction",errors='ignore')
    return dfsr

In [8]:
def get_early(y):
    fullpath = f"./Data/{y}{startpath}{y}{fallpath}{y}{finalpath}"
    dfmsf = get_measure_stars(fullpath)
    dfdsf = get_domain_stars(fullpath)
    dfsrf = get_summary_rating(fullpath)
    dff = pd.merge(dfmsf,dfdsf,on=firstfive,how='left')
    dff = pd.merge(dff,dfsrf,on=firstfive,how='left')
    dff["Year"] = f"{y}"
    dff = dff.rename(columns={f"{y} Part C Summary":"Part C Summary",f"{y} Overall":"Overall",f"{y} Part D Summary":"Part D Summary"})
    dff = dff.drop(columns = "2017 Disaster %",errors='ignore')
    return dff

In [9]:
def get_late(y):
    fullpath = f"./Data/{y}{startpath}{y}{finalpath}"
    dfmsc = get_measure_stars(fullpath)
    dfdsc = get_domain_stars(fullpath)
    dfsrc = get_summary_rating(fullpath)
    #drop disaster columns
    dfsrc = dfsrc.drop(columns=dfsrc.columns[[6,7]],axis = 1)
    df = pd.merge(dfmsc,dfdsc,on=firstfive,how='left')
    df = pd.merge(df,dfsrc,on=firstfive,how='left')
    df = df.rename(columns={f"{y} Part C Summary":"Part C Summary",f"{y} Overall":"Overall",f"{y} Part D Summary":"Part D Summary"})
    df["Year"] = f"{y}"
    return    df

In [10]:
def standardize_data():
    year = 2014
    years = [x for x in range(2015,2026)]        
    fullpath = f"./Data/{year}{startpath}{year}{fallpath}{year}{cpath}{finalpath}"
    dfmsc = get_measure_stars(fullpath)
    dfdsc = get_domain_stars(fullpath)
    dfsrc = get_summary_rating(fullpath)
    df = pd.merge(dfmsc,dfdsc,on=firstfive,how='left')
    df = pd.merge(df,dfsrc,on=firstfive,how='left')
    fullpath = f"./Data/{year}{startpath}{year}{fallpath}{year}{dpath}{finalpath}"
    dfmsd = get_measure_stars(fullpath)
    dfdsd = get_domain_stars(fullpath)
    dfsrd = get_summary_rating(fullpath)
    dfd = pd.merge(dfmsd,dfdsd,on=['Contract Number'],how='left')
    dfd = pd.merge(dfd,dfsrd,on=['Contract Number'],how='left')
    dfd = dfd.drop(columns=["Organization Type_x","Contract Name_x","Organization Marketing Name_x","Parent Organization_x","Organization Type_y","Contract Name_y","Organization Marketing Name_y","Parent Organization_y"])
    df = pd.merge(df,dfd,on=["Contract Number","Organization Type","Contract Name","Organization Marketing Name","Parent Organization","SNP"],how='left')
    df = df.rename(columns={f"{year} Part C Summary Rating":"Part C Summary",f"{year} Overall Rating":"Overall",f"{year} Part D Summary Rating":"Part D Summary"})
    df["Year"] = "2014"
    finaldf = df
    #spark = SparkSession.builder.appName("pandas to spark").getOrCreate()
    #sparkdf = spark.createDataFrame(finaldf)
    #sparkdf.show()
    for year in years:
        if (year <2020):
            df = get_early(year)
        else:
            df = get_late(year)
        #placeholderdf = spark.createDataFrame(df)
        #placeholderdf.show()
        newdf = pd.concat([finaldf,df],axis=0,join='outer')
        finaldf=newdf
    finaldf = finaldf.reset_index(drop=True)
    return finaldf

In [11]:
def clean_data(combodf1):
    for x in range(len(combodf1.keys())):
        combodf1[combodf1.keys()[x]] = combodf1[combodf1.keys()[x]].astype(str)
    combodf1 = combodf1.replace('\\s*Plan\\s*too\\s*new\\s*to\\s*be\\s*measured\\s*','Plan too new to be measured',regex=True)
    combodf1 = combodf1.replace('\\s*Plan\\s*too\\s*small\\s*to\\s*be\\s*measured\\s*','Plan too small to be measured',regex=True)
    combodf1 = combodf1.replace('\\s*Plan\\s*not\\s*required\\s*to\\s*report\\s*measure\\s*','Plan not required to report measure',regex=True)
    combodf1 = combodf1.replace('\\s*No\\s*data\\s*available\\s*','No data available',regex=True)
    combodf1 = combodf1.replace('Nodata available','No data available',regex=True)
    combodf1 = combodf1.replace('\\s*Not\\s*enough\\s*data\\s*available\\s*','Not enough data available',regex=True)
    combodf1 = combodf1.replace('\\s*Yes\\s*',True,regex=True)    
    combodf1 = combodf1.replace('\\s+No\\s+',False,regex=True)
    combodf1 = combodf1.replace('\\s+No',False,regex=True)
    combodf1 = combodf1.replace('No\\s+',False,regex=True)
    combodf1 = combodf1.replace('nan','0')

    combodf1 = combodf1.replace('\\s*Benefit\\s*not\\s*offered\\s*by\\s*plan\\s*','Benefit not offered by plan',regex=True)        
    return combodf1

In [ ]:
def cull_data(df):
    

In [12]:
combodf = standardize_data()
combodf = clean_data(combodf)
combodf = combodf.replace('nan','0')

In [37]:
combodf['Overall'].value_counts()

Overall
3.5                            1582
Plan too new to be measured    1571
4                              1320
Not enough data available      1285
3                               914
4.5                             845
Not Applicable                  804
5                               297
2.5                             192
2                                15
0                                13
Name: count, dtype: int64

In [ ]:
combodf = combodf.replace('nan','0')
combodf.to_csv("./Data/CombinedDataNoCull.csv",header=True,index=False)

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Cardiovascular Care – Cholesterol Screening,C Diabetes Care – Cholesterol Screening,C Glaucoma Testing,...,D Medication Adherence for Diabetes Medications,D Medication Adherence for Hypertension (RAS antagonists),D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR,C Medication Reconciliation Post-Discharge,C Statin Therapy for Patients with Cardiovascular Disease,D Statin Use in Persons with Diabetes (SUPD),C Controlling High Blood Pressure,C Transitions of Care,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions
0,H0022,Demo,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Buckeye Community Health Plan - MyCareOhio,Centene Corporation,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,...,0,0,0,0,0,0,0,0,0,0
1,H0028,Local CCP,"CHA HMO, INC.","CHA HMO, Inc.",Humana Inc.,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,...,0,0,0,0,0,0,0,0,0,0
2,H0084,Local CCP,CARE IMPROVEMENT PLUS OF TEXAS INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",2,2,4,4,2,...,0,0,0,0,0,0,0,0,0,0
3,H0104,Local CCP,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Blue Advantage (PPO),BlueCross BlueShield of Alabama,3,4,5,4,2,...,0,0,0,0,0,0,0,0,0,0
4,H0107,Local CCP,"BLUE CROSS AND BLUE SHIELD OF MONTANA, INC.",Blue Cross and Blue Shield of Montana,"BLUE CROSS AND BLUE SHIELD OF MONTANA, INC.",Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8833,S6875,PDP,QCC INSURANCE COMPANY,Independence Blue Cross,"Independence Health Group, Inc.",Plan not required to report measure,Plan not required to report measure,0,0,0,...,2,2,2,5,Plan not required to report measure,Plan not required to report measure,5,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
8834,S6946,PDP,CLEAR SPRING HEALTH INSURANCE COMPANY,Clear Spring Health,Group 1001,Plan not required to report measure,Plan not required to report measure,0,0,0,...,1,1,1,2,Plan not required to report measure,Plan not required to report measure,2,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
8835,S8067,PDP,Avalon Insurance Company,Avalon Insurance Company,CAPITAL BLUE CROSS,Plan not required to report measure,Plan not required to report measure,0,0,0,...,4,4,4,4,Plan not required to report measure,Plan not required to report measure,4,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
8836,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",Plan not required to report measure,Plan not required to report measure,0,0,0,...,3,2,2,3,Plan not required to report measure,Plan not required to report measure,3,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure


In [16]:
combodf = standardize_data()
combodf.to_csv("./Data/CombinedData.csv",header=True,index=False)

In [17]:
combodf = clean_data(combodf)
combodf.to_csv("./Data/CombinedDataCleaned.csv",header=True,index=False)

C:\Users\samsi\AppData\Local\Temp\ipykernel_37080\2318447181.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combodf1 = combodf1.replace('\\s*No\\s*',False,regex=True)


In [79]:
combodf.loc[combodf['C Breast Cancer Screening']=='Plan too small to be measured']

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Cardiovascular Care – Cholesterol Screening,C Diabetes Care – Cholesterol Screening,C Glaucoma Testing,...,D Medication Adherence for Diabetes Medications,D Medication Adherence for Hypertension (RAS antagonists),D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR,C Medication Reconciliation Post-Discharge,C Statin Therapy for Patients with Cardiovascular Disease,D Statin Use in Persons with Diabetes (SUPD),C Controlling High Blood Pressure,C Transitions of Care,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions
8,H0141,Local CCP,"MCLAREN HEALTH PLAN, INC.",McLarenAdvantage,McLaren Health Care Corporation,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
18,H0294,Local CCP,CARE IMPROVEMENT PLUS WISCONSIN INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
31,H0408,Local CCP,UNITEDHEALTHCARE INSURANCE COMPANY,UnitedHealthcare,"UnitedHealth Group, Inc.",Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
36,H0490,Local CCP,"MOLINA HEALTHCARE OF OHIO, INC.",Molina Healthcare of Ohio,"Molina Healthcare, Inc.,",Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
37,H0502,1876 Cost,THE CONTRA COSTA HEALTH PLAN,Contra Costa Health Plan,Contra Costa Health Services,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12645,H9589,Local CCP,"CLEAR SPRING HEALTH (GA), INC.",Clear Spring Health,Group 1001,Plan too small to be measured,Plan too small to be measured,nan,nan,nan,...,Not enough data available,Not enough data available,Not enough data available,Not enough data available,Plan too small to be measured,Plan too small to be measured,Not enough data available,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured
12662,H9826,Local CCP,"COMMUNITY HEALTH CHOICE TEXAS, INC.",Community Health Choice,Harris County Hospital District,Plan too small to be measured,Plan too small to be measured,nan,nan,nan,...,2,2,1,2,Plan too small to be measured,Plan too small to be measured,4,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured
12663,H9827,Local CCP,"MOUNT CARMEL HEALTH PLAN OF NEW YORK, INC.",Trinity Health Plan New York,Trinity Health Corporation,Plan too small to be measured,Plan too small to be measured,nan,nan,nan,...,4,5,4,Not enough data available,Plan too small to be measured,Plan too small to be measured,5,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured
12667,H9888,Local CCP,DEVOTED HEALTH INSURANCE COMPANY OF ALABAMA INC,Devoted Health,"Devoted Health, Inc.",Plan too small to be measured,Plan too small to be measured,nan,nan,nan,...,1,2,2,Not enough data available,Plan too small to be measured,Plan too small to be measured,3,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured


In [18]:
combodf2 = combodf.copy()

rowswithnodata = []
datatosearch = ['1','2','3','4','5']
for i, r in combodf2.iterrows():
    row = r.to_list()
    if(any(val in row for val in datatosearch)):
        continue
    else:
        rowswithnodata.append(i)
rowswithnodata
#combodf2 = combodf2.replace('Not enough data available',0)


[0,
 4,
 7,
 9,
 10,
 14,
 17,
 28,
 34,
 35,
 57,
 58,
 60,
 65,
 86,
 95,
 108,
 116,
 119,
 122,
 123,
 127,
 130,
 134,
 136,
 138,
 162,
 180,
 181,
 182,
 184,
 196,
 197,
 200,
 205,
 206,
 216,
 217,
 218,
 219,
 222,
 231,
 234,
 260,
 265,
 266,
 270,
 287,
 308,
 310,
 317,
 318,
 327,
 332,
 342,
 350,
 367,
 369,
 375,
 385,
 391,
 403,
 407,
 408,
 427,
 451,
 469,
 473,
 505,
 510,
 511,
 512,
 514,
 518,
 522,
 523,
 525,
 526,
 527,
 531,
 533,
 536,
 539,
 540,
 544,
 545,
 548,
 550,
 555,
 559,
 561,
 566,
 567,
 568,
 569,
 574,
 575,
 576,
 577,
 579,
 581,
 585,
 586,
 588,
 589,
 590,
 591,
 592,
 594,
 595,
 596,
 597,
 598,
 600,
 601,
 603,
 605,
 607,
 610,
 614,
 615,
 617,
 618,
 630,
 631,
 632,
 633,
 634,
 635,
 638,
 641,
 644,
 648,
 649,
 650,
 651,
 652,
 654,
 656,
 659,
 661,
 662,
 665,
 683,
 687,
 689,
 691,
 692,
 696,
 698,
 707,
 714,
 716,
 736,
 737,
 739,
 744,
 760,
 761,
 784,
 791,
 796,
 797,
 801,
 804,
 808,
 809,
 811,
 813,
 823,


In [19]:
rowswithnodata.reverse()
for row in rowswithnodata:
    combodf2 = combodf2.drop(combodf2.index[row])
combodf2

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Cardiovascular Care – Cholesterol Screening,C Diabetes Care – Cholesterol Screening,C Glaucoma Testing,...,D Medication Adherence for Diabetes Medications,D Medication Adherence for Hypertension (RAS antagonists),D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR,C Medication Reconciliation Post-Discharge,C Statin Therapy for Patients with Cardiovascular Disease,D Statin Use in Persons with Diabetes (SUPD),C Controlling High Blood Pressure,C Transitions of Care,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions
1,H0028,Local CCP,"CHA HMO, INC.","CHA HMO, Inc.",Humana Inc.,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,H0084,Local CCP,CARE IMPROVEMENT PLUS OF TEXAS INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",2,2,4,4,2,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,H0104,Local CCP,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Blue Advantage (PPO),BlueCross BlueShield of Alabama,3,4,5,4,2,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,H0108,Local CCP,"HUMANA HEALTH PLAN OF CALIFORNIA, INC.","Humana Health Plan of California, Inc.",Humana Inc.,3,3,5,4,2,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,H0117,Local CCP,"WELLCARE OF OHIO, INC.",WellCare,"WellCare Health Plans, Inc.",3,4,4,4,3,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8833,S6875,PDP,QCC INSURANCE COMPANY,Independence Blue Cross,"Independence Health Group, Inc.",Plan not required to report measure,Plan not required to report measure,nan,nan,nan,...,2,2,2,5,Plan not required to report measure,Plan not required to report measure,5,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
8834,S6946,PDP,CLEAR SPRING HEALTH INSURANCE COMPANY,Clear Spring Health,Group 1001,Plan not required to report measure,Plan not required to report measure,nan,nan,nan,...,1,1,1,2,Plan not required to report measure,Plan not required to report measure,2,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
8835,S8067,PDP,Avalon Insurance Company,Avalon Insurance Company,CAPITAL BLUE CROSS,Plan not required to report measure,Plan not required to report measure,nan,nan,nan,...,4,4,4,4,Plan not required to report measure,Plan not required to report measure,4,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
8836,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",Plan not required to report measure,Plan not required to report measure,nan,nan,nan,...,3,2,2,3,Plan not required to report measure,Plan not required to report measure,3,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure


In [20]:
combodf2 = combodf2.replace('nan','0')
combodf2

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Cardiovascular Care – Cholesterol Screening,C Diabetes Care – Cholesterol Screening,C Glaucoma Testing,...,D Medication Adherence for Diabetes Medications,D Medication Adherence for Hypertension (RAS antagonists),D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR,C Medication Reconciliation Post-Discharge,C Statin Therapy for Patients with Cardiovascular Disease,D Statin Use in Persons with Diabetes (SUPD),C Controlling High Blood Pressure,C Transitions of Care,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions
1,H0028,Local CCP,"CHA HMO, INC.","CHA HMO, Inc.",Humana Inc.,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,...,0,0,0,0,0,0,0,0,0,0
2,H0084,Local CCP,CARE IMPROVEMENT PLUS OF TEXAS INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",2,2,4,4,2,...,0,0,0,0,0,0,0,0,0,0
3,H0104,Local CCP,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Blue Advantage (PPO),BlueCross BlueShield of Alabama,3,4,5,4,2,...,0,0,0,0,0,0,0,0,0,0
5,H0108,Local CCP,"HUMANA HEALTH PLAN OF CALIFORNIA, INC.","Humana Health Plan of California, Inc.",Humana Inc.,3,3,5,4,2,...,0,0,0,0,0,0,0,0,0,0
6,H0117,Local CCP,"WELLCARE OF OHIO, INC.",WellCare,"WellCare Health Plans, Inc.",3,4,4,4,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8833,S6875,PDP,QCC INSURANCE COMPANY,Independence Blue Cross,"Independence Health Group, Inc.",Plan not required to report measure,Plan not required to report measure,0,0,0,...,2,2,2,5,Plan not required to report measure,Plan not required to report measure,5,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
8834,S6946,PDP,CLEAR SPRING HEALTH INSURANCE COMPANY,Clear Spring Health,Group 1001,Plan not required to report measure,Plan not required to report measure,0,0,0,...,1,1,1,2,Plan not required to report measure,Plan not required to report measure,2,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
8835,S8067,PDP,Avalon Insurance Company,Avalon Insurance Company,CAPITAL BLUE CROSS,Plan not required to report measure,Plan not required to report measure,0,0,0,...,4,4,4,4,Plan not required to report measure,Plan not required to report measure,4,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
8836,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",Plan not required to report measure,Plan not required to report measure,0,0,0,...,3,2,2,3,Plan not required to report measure,Plan not required to report measure,3,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure


In [21]:
combodf2 = combodf2.replace('Plan too new to be measured',np.nan)
combodf2 = combodf2.dropna(how='any')
combodf2

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Cardiovascular Care – Cholesterol Screening,C Diabetes Care – Cholesterol Screening,C Glaucoma Testing,...,D Medication Adherence for Diabetes Medications,D Medication Adherence for Hypertension (RAS antagonists),D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR,C Medication Reconciliation Post-Discharge,C Statin Therapy for Patients with Cardiovascular Disease,D Statin Use in Persons with Diabetes (SUPD),C Controlling High Blood Pressure,C Transitions of Care,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions
2,H0084,Local CCP,CARE IMPROVEMENT PLUS OF TEXAS INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",2,2,4,4,2,...,0,0,0,0,0,0,0,0,0,0
3,H0104,Local CCP,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Blue Advantage (PPO),BlueCross BlueShield of Alabama,3,4,5,4,2,...,0,0,0,0,0,0,0,0,0,0
5,H0108,Local CCP,"HUMANA HEALTH PLAN OF CALIFORNIA, INC.","Humana Health Plan of California, Inc.",Humana Inc.,3,3,5,4,2,...,0,0,0,0,0,0,0,0,0,0
6,H0117,Local CCP,"WELLCARE OF OHIO, INC.",WellCare,"WellCare Health Plans, Inc.",3,4,4,4,3,...,0,0,0,0,0,0,0,0,0,0
8,H0141,Local CCP,"MCLAREN HEALTH PLAN, INC.",McLarenAdvantage,McLaren Health Care Corporation,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8833,S6875,PDP,QCC INSURANCE COMPANY,Independence Blue Cross,"Independence Health Group, Inc.",Plan not required to report measure,Plan not required to report measure,0,0,0,...,2,2,2,5,Plan not required to report measure,Plan not required to report measure,5,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
8834,S6946,PDP,CLEAR SPRING HEALTH INSURANCE COMPANY,Clear Spring Health,Group 1001,Plan not required to report measure,Plan not required to report measure,0,0,0,...,1,1,1,2,Plan not required to report measure,Plan not required to report measure,2,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
8835,S8067,PDP,Avalon Insurance Company,Avalon Insurance Company,CAPITAL BLUE CROSS,Plan not required to report measure,Plan not required to report measure,0,0,0,...,4,4,4,4,Plan not required to report measure,Plan not required to report measure,4,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
8836,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",Plan not required to report measure,Plan not required to report measure,0,0,0,...,3,2,2,3,Plan not required to report measure,Plan not required to report measure,3,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure


In [22]:
combodf2 = combodf2.replace('Plan too small to be measured','0')
combodf2 = combodf2.replace('Plan not required to report measure','0')
combodf2 = combodf2.replace('Plan not required to report ','0')
combodf2 = combodf2.replace('Nodata available','0')
combodf2 = combodf2.replace('Not enough data available','0')
combodf2 = combodf2.replace('Benefit not offered by plan','0')

In [19]:
combodf2.to_csv('./Data/CombinedDataSet0.csv',header=True,index=False)

In [28]:
badsnp = combodf2.loc[combodf['SNP']=='nan']
badsnp.to_csv("./Data/badSNP.csv",header=True,index=False)

In [23]:
cols = combodf2.keys().to_list()
for col in firstfive:
    cols.remove(col)
for col in lasttwo:
    cols.remove(col)
newkeys = firstfive+cols+lasttwo
newkeys

['Contract Number',
 'Organization Type',
 'Contract Name',
 'Organization Marketing Name',
 'Parent Organization',
 'C Breast Cancer Screening',
 'C Colorectal Cancer Screening',
 'C Cardiovascular Care – Cholesterol Screening',
 'C Diabetes Care – Cholesterol Screening',
 'C Glaucoma Testing',
 'C Annual Flu Vaccine',
 'C Improving or Maintaining Physical Health',
 'C Improving or Maintaining Mental Health',
 'C Monitoring Physical Activity',
 'C Adult BMI Assessment',
 'C Care for Older Adults – Medication Review',
 'C Care for Older Adults – Functional Status Assessment',
 'C Care for Older Adults – Pain Screening',
 'C Osteoporosis Management in Women who had a Fracture',
 'C Diabetes Care – Eye Exam',
 'C Diabetes Care – Kidney Disease Monitoring',
 'C Diabetes Care – Blood Sugar Controlled',
 'C Diabetes Care – Cholesterol Controlled',
 'C Controlling Blood Pressure',
 'C Rheumatoid Arthritis Management',
 'C Improving Bladder Control',
 'C Reducing the Risk of Falling',
 'C Pla

In [24]:
newdf = combodf2[newkeys]
newdf

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Cardiovascular Care – Cholesterol Screening,C Diabetes Care – Cholesterol Screening,C Glaucoma Testing,...,D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR,C Medication Reconciliation Post-Discharge,C Statin Therapy for Patients with Cardiovascular Disease,D Statin Use in Persons with Diabetes (SUPD),C Controlling High Blood Pressure,C Transitions of Care,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions,Year,Overall
2,H0084,Local CCP,CARE IMPROVEMENT PLUS OF TEXAS INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",2,2,4,4,2,...,0,0,0,0,0,0,0,0,2014,3
3,H0104,Local CCP,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Blue Advantage (PPO),BlueCross BlueShield of Alabama,3,4,5,4,2,...,0,0,0,0,0,0,0,0,2014,3
5,H0108,Local CCP,"HUMANA HEALTH PLAN OF CALIFORNIA, INC.","Humana Health Plan of California, Inc.",Humana Inc.,3,3,5,4,2,...,0,0,0,0,0,0,0,0,2014,3.5
6,H0117,Local CCP,"WELLCARE OF OHIO, INC.",WellCare,"WellCare Health Plans, Inc.",3,4,4,4,3,...,0,0,0,0,0,0,0,0,2014,3
8,H0141,Local CCP,"MCLAREN HEALTH PLAN, INC.",McLarenAdvantage,McLaren Health Care Corporation,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2014,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8833,S6875,PDP,QCC INSURANCE COMPANY,Independence Blue Cross,"Independence Health Group, Inc.",0,0,0,0,0,...,2,5,0,0,5,0,0,0,2025,False
8834,S6946,PDP,CLEAR SPRING HEALTH INSURANCE COMPANY,Clear Spring Health,Group 1001,0,0,0,0,0,...,1,2,0,0,2,0,0,0,2025,False
8835,S8067,PDP,Avalon Insurance Company,Avalon Insurance Company,CAPITAL BLUE CROSS,0,0,0,0,0,...,4,4,0,0,4,0,0,0,2025,False
8836,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",0,0,0,0,0,...,2,3,0,0,3,0,0,0,2025,False


In [25]:
for col in cols:
    newdf[col] = newdf[col].astype(float)



In [26]:
newdf.to_csv("./Data/CombinedDataSet0ChangedToint.csv")

In [20]:
len(combodf2)

12746

In [19]:
len(rowswithnodata)

2091

In [13]:
def add_non_numeric_cols(item):
    try:
        val = float(item)
        return "Numeric"
    except:
        return item

In [14]:
check = add_non_numeric_cols("different words")
check

'different words'

In [15]:
def change_to_nums(item):
    match item:
        case "1":
            return int(item)
        case "2":
            return int(item)
        case "3":
            return int(item)
        case "4":
            return int(item)
        case "5":
            return int(item)
        case _:
            return 0

In [16]:
def change_to_dec(item):
    try:
        val = float(item)
        return val
    except:
        return 0

In [17]:
combodf4 = combodf.copy()
cols = combodf4.keys().to_list()
cols.append("Overall")
for col in firstfive:
    cols.remove(col)
for col in lasttwo:
    cols.remove(col)
for col in cols:
    combodf4[col+" Non-numeric"] = combodf4[col].apply(add_non_numeric_cols)
    combodf4[col]=combodf4[col].apply(change_to_dec)
    
combodf4["Overall"].value_counts()

C:\Users\samsi\AppData\Local\Temp\ipykernel_11488\1015568200.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combodf4[col+" Non-numeric"] = combodf4[col].apply(add_non_numeric_cols)
C:\Users\samsi\AppData\Local\Temp\ipykernel_11488\1015568200.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combodf4[col+" Non-numeric"] = combodf4[col].apply(add_non_numeric_cols)
C:\Users\samsi\AppData\Local\Temp\ipykernel_11488\1015568200.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fra

Overall
0.0    3673
3.5    1582
4.0    1320
3.0     914
4.5     845
5.0     297
2.5     192
2.0      15
Name: count, dtype: int64

In [18]:
newcols = []
for col in cols:
    newcols.append(col)
    newcols.append(col+" Non-numeric")
finalcols =[]
for col in firstfive:
    finalcols.append(col)
finalcols.append("Year")
for col in newcols:
    finalcols.append(col)

newdf1 = combodf4[finalcols]
newdf1

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,Year,C Breast Cancer Screening,C Breast Cancer Screening Non-numeric,C Colorectal Cancer Screening,C Colorectal Cancer Screening Non-numeric,...,D Statin Use in Persons with Diabetes (SUPD),D Statin Use in Persons with Diabetes (SUPD) Non-numeric,C Controlling High Blood Pressure,C Controlling High Blood Pressure Non-numeric,C Transitions of Care,C Transitions of Care Non-numeric,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions Non-numeric,Overall,Overall Non-numeric
0,H0022,Demo,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Buckeye Community Health Plan - MyCareOhio,Centene Corporation,2014,0.0,Plan too new to be measured,0.0,Plan too new to be measured,...,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Plan too new to be measured
1,H0028,Local CCP,"CHA HMO, INC.","CHA HMO, Inc.",Humana Inc.,2014,0.0,Plan too new to be measured,0.0,Plan too new to be measured,...,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Plan too new to be measured
2,H0084,Local CCP,CARE IMPROVEMENT PLUS OF TEXAS INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",2014,2.0,Numeric,2.0,Numeric,...,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,3.0,Numeric
3,H0104,Local CCP,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Blue Advantage (PPO),BlueCross BlueShield of Alabama,2014,3.0,Numeric,4.0,Numeric,...,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,3.0,Numeric
4,H0107,Local CCP,"BLUE CROSS AND BLUE SHIELD OF MONTANA, INC.",Blue Cross and Blue Shield of Montana,"BLUE CROSS AND BLUE SHIELD OF MONTANA, INC.",2014,0.0,Plan too new to be measured,0.0,Plan too new to be measured,...,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Plan too new to be measured
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8833,S6875,PDP,QCC INSURANCE COMPANY,Independence Blue Cross,"Independence Health Group, Inc.",2025,0.0,Plan not required to report measure,0.0,Plan not required to report measure,...,5.0,Numeric,0.0,Plan not required to report measure,0.0,Plan not required to report measure,0.0,Plan not required to report measure,0.0,Not Applicable
8834,S6946,PDP,CLEAR SPRING HEALTH INSURANCE COMPANY,Clear Spring Health,Group 1001,2025,0.0,Plan not required to report measure,0.0,Plan not required to report measure,...,2.0,Numeric,0.0,Plan not required to report measure,0.0,Plan not required to report measure,0.0,Plan not required to report measure,0.0,Not Applicable
8835,S8067,PDP,Avalon Insurance Company,Avalon Insurance Company,CAPITAL BLUE CROSS,2025,0.0,Plan not required to report measure,0.0,Plan not required to report measure,...,4.0,Numeric,0.0,Plan not required to report measure,0.0,Plan not required to report measure,0.0,Plan not required to report measure,0.0,Not Applicable
8836,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",2025,0.0,Plan not required to report measure,0.0,Plan not required to report measure,...,3.0,Numeric,0.0,Plan not required to report measure,0.0,Plan not required to report measure,0.0,Plan not required to report measure,0.0,Not Applicable


In [101]:
newdf1.loc[newdf1['Year']<2024]

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,Year,C Breast Cancer Screening,C Breast Cancer Screening Non-numeric,C Colorectal Cancer Screening,C Colorectal Cancer Screening Non-numeric,...,D Statin Use in Persons with Diabetes (SUPD),D Statin Use in Persons with Diabetes (SUPD) Non-numeric,C Controlling High Blood Pressure,C Controlling High Blood Pressure Non-numeric,C Transitions of Care,C Transitions of Care Non-numeric,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions Non-numeric,Overall,Overall Non-numeric
0,H0022,Demo,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Buckeye Community Health Plan - MyCareOhio,Centene Corporation,2014,0.0,Plan too new to be measured,0.0,Plan too new to be measured,...,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Plan too new to be measured
1,H0028,Local CCP,"CHA HMO, INC.","CHA HMO, Inc.",Humana Inc.,2014,0.0,Plan too new to be measured,0.0,Plan too new to be measured,...,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Plan too new to be measured
2,H0084,Local CCP,CARE IMPROVEMENT PLUS OF TEXAS INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",2014,2.0,Numeric,2.0,Numeric,...,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,3.0,Numeric
3,H0104,Local CCP,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Blue Advantage (PPO),BlueCross BlueShield of Alabama,2014,3.0,Numeric,4.0,Numeric,...,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,3.0,Numeric
4,H0107,Local CCP,"BLUE CROSS AND BLUE SHIELD OF MONTANA, INC.",Blue Cross and Blue Shield of Montana,"BLUE CROSS AND BLUE SHIELD OF MONTANA, INC.",2014,0.0,Plan too new to be measured,0.0,Plan too new to be measured,...,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Plan too new to be measured
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7187,S8182,PDP,AMERIGROUP INSURANCE COMPANY,Amerigroup,"Elevance Health, Inc.",2023,0.0,Plan not required to report measure,0.0,Plan not required to report measure,...,1.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Not Applicable
7188,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",2023,0.0,Plan not required to report measure,0.0,Plan not required to report measure,...,3.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Not Applicable
7189,S9409,PDP,"VERMONT BLUE ADVANTAGE, INC.",Vermont Blue Advantage,Blue Cross Blue Shield of Michigan Mutual Ins....,2023,0.0,Plan not required to report measure,0.0,Plan not required to report measure,...,0.0,Plan too new to be measured,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Not Applicable
7190,S9701,PDP,"DEAN HEALTH INSURANCE, INC.",Navitus MedicareRx,Medica Holding Company,2023,0.0,Plan not required to report measure,0.0,Plan not required to report measure,...,4.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Not Applicable


In [29]:
testcols = newdf1.keys().to_list()
newtestcols = []
for col in testcols:
    if('Non-numeric' in col):
        newtestcols.append(col)
newtestcols

['C Breast Cancer Screening Non-numeric',
 'C Colorectal Cancer Screening Non-numeric',
 'C Cardiovascular Care – Cholesterol Screening Non-numeric',
 'C Diabetes Care – Cholesterol Screening Non-numeric',
 'C Glaucoma Testing Non-numeric',
 'C Annual Flu Vaccine Non-numeric',
 'C Improving or Maintaining Physical Health Non-numeric',
 'C Improving or Maintaining Mental Health Non-numeric',
 'C Monitoring Physical Activity Non-numeric',
 'C Adult BMI Assessment Non-numeric',
 'C Care for Older Adults – Medication Review Non-numeric',
 'C Care for Older Adults – Functional Status Assessment Non-numeric',
 'C Care for Older Adults – Pain Screening Non-numeric',
 'C Osteoporosis Management in Women who had a Fracture Non-numeric',
 'C Diabetes Care – Eye Exam Non-numeric',
 'C Diabetes Care – Kidney Disease Monitoring Non-numeric',
 'C Diabetes Care – Blood Sugar Controlled Non-numeric',
 'C Diabetes Care – Cholesterol Controlled Non-numeric',
 'C Controlling Blood Pressure Non-numeric',


In [39]:
newdf1.to_csv("./Data/CombinedDataNoEnrollment.csv",header=True,index=False)

In [36]:
check = change_to_nums("words")
check

0

In [ ]:
combodf3 = combodf.copy()
combodf3['C Cardiovascular Care – Cholesterol Screening_Object'] = combodf3['C Cardiovascular Care – Cholesterol Screening'].apply(add_non_numeric_cols())

0       Plan too new to be measured
1       Plan too new to be measured
2                                 4
3                                 5
4       Plan too new to be measured
                   ...             
8833                            nan
8834                            nan
8835                            nan
8836                            nan
8837                            nan
Name: C Cardiovascular Care – Cholesterol Screening, Length: 8838, dtype: object

In [17]:
combodf3['C Cardiovascular Care – Cholesterol Screening_Object'] = combodf3['C Cardiovascular Care – Cholesterol Screening'].apply(add_non_numeric_cols)
combodf3['C Cardiovascular Care – Cholesterol Screening_Object']

0       Plan too new to be measured
1       Plan too new to be measured
2                           Numeric
3                           Numeric
4       Plan too new to be measured
                   ...             
8833                            nan
8834                            nan
8835                            nan
8836                            nan
8837                            nan
Name: C Cardiovascular Care – Cholesterol Screening_Object, Length: 8838, dtype: object

In [33]:
enrollmentcsv = pd.read_excel("./Data/Enrollment.xlsx")
statepop = pd.read_csv("./Data/StatePopulations.csv")

In [52]:
abfips = us.states.mapping('fips','abbr')
abfips = pd.DataFrame.from_dict([abfips])
abfips = abfips.transpose()
abfips = abfips.reset_index()
abfips = abfips.rename(columns={"index":"fips",0:"abbr"})
print(abfips)
enrollab = pd.merge(enrollmentcsv,abfips,left_on='State',right_on='abbr')
enrollab = enrollab.drop(columns="abbr")
enrollab['fips'] = enrollab['fips'].astype(int)
enrollabpop = pd.merge(enrollab,statepop,left_on=['fips','Year'],right_on=['state','Year'])
enrollabpop = enrollabpop.drop(columns=['state'])
enrollabpop

   fips abbr
0    01   AL
1    02   AK
2    04   AZ
3    05   AR
4    06   CA
5    08   CO
6    09   CT
7    10   DE
8    12   FL
9    13   GA
10   15   HI
11   16   ID
12   17   IL
13   18   IN
14   19   IA
15   20   KS
16   21   KY
17   22   LA
18   23   ME
19   24   MD
20   25   MA
21   26   MI
22   27   MN
23   28   MS
24   29   MO
25   30   MT
26   31   NE
27   32   NV
28   33   NH
29   34   NJ
30   35   NM
31   36   NY
32   37   NC
33   38   ND
34   39   OH
35   40   OK
36   41   OR
37   42   PA
38   44   RI
39   45   SC
40   46   SD
41   47   TN
42   48   TX
43   49   UT
44   50   VT
45   51   VA
46   53   WA
47   54   WV
48   55   WI
49   56   WY
50   60   AS
51   66   GU
52   69   MP
53   72   PR
54   78   VI


,Contract Number,State,Year,Month,Total Enrollments by State,fips,NAME,Population Over 65
0,H1036,FL,2014,1,303727,12,Florida,396346.0
1,H0524,OR,2014,1,34,41,Oregon,79339.0
2,E2332,CO,2014,1,223,8,Colorado,111882.0
3,H0544,CA,2014,1,51036,6,California,882861.0
4,H0084,NM,2014,1,3003,35,New Mexico,47140.0
...,...,...,...,...,...,...,...,...
300333,S8841,ND,2023,12,260,38,North Dakota,19344.0
300334,S8841,IL,2023,12,9558,17,Illinois,251673.0
300335,S8841,OH,2023,12,8835,39,Ohio,237749.0
300336,S7126,FL,2023,12,3009,12,Florida,385572.0


In [69]:
enrollabpop1 = enrollabpop

In [73]:
testgroup = pd.DataFrame(enrollabpop.groupby(['Contract Number','State','Year'])['Total Enrollments by State'].mean())
testgroup

Total Enrollments by State
Contract Number State Year                            
E0654           AK    2014                  410.833333
                      2015                  394.500000
                      2016                  368.833333
                      2017                  351.000000
                      2018                  334.363636
...                                                ...
S9701           WV    2021                  316.333333
                      2022                  287.333333
                      2023                  262.416667
                WY    2022                   27.750000
                      2023                   29.500000

[27223 rows x 1 columns]

In [75]:
testgroup = testgroup.reset_index()

In [78]:
testgroup['Total Enrollments by State'] = testgroup['Total Enrollments by State'].round(0)
testgroup = testgroup.rename(columns={"Total Enrollments by State":"Avg Enrollment"})
testgroup

,Contract Number,State,Year,Avg Enrollment
0,E0654,AK,2014,411.0
1,E0654,AK,2015,394.0
2,E0654,AK,2016,369.0
3,E0654,AK,2017,351.0
4,E0654,AK,2018,334.0
...,...,...,...,...
27218,S9701,WV,2021,316.0
27219,S9701,WV,2022,287.0
27220,S9701,WV,2023,262.0
27221,S9701,WY,2022,28.0


In [79]:
enrollwavg = pd.merge(testgroup,enrollabpop,on=['Contract Number', 'State','Year'])
enrollwavg

,Contract Number,State,Year,Avg Enrollment,Month,Total Enrollments by State,fips,NAME,Population Over 65
0,E0654,AK,2014,411.0,1,390,2,Alaska,17868.0
1,E0654,AK,2014,411.0,2,419,2,Alaska,17868.0
2,E0654,AK,2014,411.0,3,420,2,Alaska,17868.0
3,E0654,AK,2014,411.0,4,419,2,Alaska,17868.0
4,E0654,AK,2014,411.0,5,418,2,Alaska,17868.0
...,...,...,...,...,...,...,...,...,...
300333,S9701,WY,2023,30.0,8,30,56,Wyoming,12147.0
300334,S9701,WY,2023,30.0,9,30,56,Wyoming,12147.0
300335,S9701,WY,2023,30.0,10,30,56,Wyoming,12147.0
300336,S9701,WY,2023,30.0,11,29,56,Wyoming,12147.0


In [80]:
enrollwavg["Standardized Enrollment"] = enrollwavg['Avg Enrollment']/enrollwavg['Population Over 65']
enrollwavg

,Contract Number,State,Year,Avg Enrollment,Month,Total Enrollments by State,fips,NAME,Population Over 65,Standardized Enrollment
0,E0654,AK,2014,411.0,1,390,2,Alaska,17868.0,0.023002
1,E0654,AK,2014,411.0,2,419,2,Alaska,17868.0,0.023002
2,E0654,AK,2014,411.0,3,420,2,Alaska,17868.0,0.023002
3,E0654,AK,2014,411.0,4,419,2,Alaska,17868.0,0.023002
4,E0654,AK,2014,411.0,5,418,2,Alaska,17868.0,0.023002
...,...,...,...,...,...,...,...,...,...,...
300333,S9701,WY,2023,30.0,8,30,56,Wyoming,12147.0,0.002470
300334,S9701,WY,2023,30.0,9,30,56,Wyoming,12147.0,0.002470
300335,S9701,WY,2023,30.0,10,30,56,Wyoming,12147.0,0.002470
300336,S9701,WY,2023,30.0,11,29,56,Wyoming,12147.0,0.002470


In [89]:
enrollwavg1 = enrollwavg.copy()
enrollwavg1 = enrollwavg1.drop(columns=['Month','Total Enrollments by State'])
enrollwavg1 = enrollwavg1.drop_duplicates()

In [97]:
datatomerge = enrollwavg1.groupby(["Contract Number","Year"])['Standardized Enrollment'].mean()
datatomerge = datatomerge.reset_index()
datatomerge

,Contract Number,Year,Standardized Enrollment
0,E0654,2014,0.002208
1,E0654,2015,0.002300
2,E0654,2016,0.002250
3,E0654,2017,0.002315
4,E0654,2018,0.002332
...,...,...,...
8218,S9701,2019,0.010970
8219,S9701,2020,0.012278
8220,S9701,2021,0.011875
8221,S9701,2022,0.015907


In [99]:
newdf1['Year'] = newdf1['Year'].astype(int)

In [116]:
finaldf = pd.merge(newdf1,datatomerge,on=['Contract Number','Year'])
finaldf

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,Year,C Breast Cancer Screening,C Breast Cancer Screening Non-numeric,C Colorectal Cancer Screening,C Colorectal Cancer Screening Non-numeric,...,D Statin Use in Persons with Diabetes (SUPD) Non-numeric,C Controlling High Blood Pressure,C Controlling High Blood Pressure Non-numeric,C Transitions of Care,C Transitions of Care Non-numeric,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions Non-numeric,Overall,Overall Non-numeric,Standardized Enrollment
0,H0022,Demo,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Buckeye Community Health Plan - MyCareOhio,Centene Corporation,2014,0.0,Plan too new to be measured,0.0,Plan too new to be measured,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Plan too new to be measured,0.005863
1,H0028,Local CCP,"CHA HMO, INC.","CHA HMO, Inc.",Humana Inc.,2014,0.0,Plan too new to be measured,0.0,Plan too new to be measured,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Plan too new to be measured,0.047065
2,H0084,Local CCP,CARE IMPROVEMENT PLUS OF TEXAS INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",2014,2.0,Numeric,2.0,Numeric,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,3.0,Numeric,0.031054
3,H0104,Local CCP,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Blue Advantage (PPO),BlueCross BlueShield of Alabama,2014,3.0,Numeric,4.0,Numeric,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,3.0,Numeric,0.209081
4,H0107,Local CCP,"BLUE CROSS AND BLUE SHIELD OF MONTANA, INC.",Blue Cross and Blue Shield of Montana,"BLUE CROSS AND BLUE SHIELD OF MONTANA, INC.",2014,0.0,Plan too new to be measured,0.0,Plan too new to be measured,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Plan too new to be measured,0.005071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4425,S7694,PDP,Envision Insurance Company,EnvisionInsurance,Rite Aid Corporation,2020,0.0,Plan not required to report measure,0.0,Plan not required to report measure,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Not Applicable,0.138736
4426,S8067,PDP,Avalon Insurance Company,Avalon Insurance Company,Capital BlueCross,2020,0.0,Plan not required to report measure,0.0,Plan not required to report measure,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Not Applicable,0.004767
4427,S8182,PDP,Amerigroup Insurance Company,Amerivantage Rx (PDP),Anthem Inc.,2020,0.0,Plan not required to report measure,0.0,Plan not required to report measure,...,Plan too new to be measured,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Not Applicable,0.009767
4428,S8841,PDP,"Optum Insurance Of Ohio, Inc.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",2020,0.0,Plan not required to report measure,0.0,Plan not required to report measure,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Not Applicable,0.105511


In [117]:
def change_enrolled(item):
    if item <.05:
        return 1
    elif item < .3:
        return 2
    elif item < .7:
        return 3
    else:
        return 4

In [118]:
finaldf["Standardized Enrollment"] = finaldf["Standardized Enrollment"].apply(change_enrolled)
finaldf['Standardized Enrollment'].value_counts()

Standardized Enrollment
1    2573
2    1485
3     301
4      71
Name: count, dtype: int64

In [119]:
finaldf.to_csv("./Data/FinalData.csv",header=True,index=False)